In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.7 MB/s eta 0:00:00


In [ ]:
!pip install torch-cluster

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torch-cluster: filename=torch_cluster-1.6.3-cp310-cp310-linux_x86_64.whl size=2021620 sha256=2b5a8adeaf0b096c686eea94c0dee8d44abec31611677bae9fe57aa1eed6b99a
  Stored in directory: /root/.cache/pip/wheels/51/78/c3/536637b3cdcc3313aa5e8851a6c72b97f6a01877e68c7595e3
Successfully built torch-cluster


In [ ]:
from torch_cluster import knn_graph, knn

In [ ]:
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.1.0+cpu.html


Looking in links: https://data.pyg.org/whl/torch-2.1.0+cpu.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.3/497.3 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.5/208.5 kB 2.8 MB/s eta 0:00:00


In [ ]:
# !pip install torch_xla


In [ ]:
import random
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import DynamicEdgeConv, MessagePassing, knn_graph, GCNConv
from torch.nn import Sequential as Seq, Linear, ReLU, BatchNorm1d
from torch_geometric.utils import softmax


import torch
# import torch_xla
# import torch_xla.core.xla_model as xm


from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, mean_squared_error, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, OrdinalEncoder, Normalizer, LabelEncoder

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/UJIINDOORLOC/TrainingData.csv')
df_val = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/UJIINDOORLOC/ValidationData.csv')

train_numb = len(df_train)
test_numb = len(df_val)

df = pd.concat([df_train, df_val])
df

,WAP001,WAP002,WAP003,WAP004,WAP005,WAP006,WAP007,WAP008,WAP009,WAP010,...,WAP520,LONGITUDE,LATITUDE,FLOOR,BUILDINGID,SPACEID,RELATIVEPOSITION,USERID,PHONEID,TIMESTAMP
0,100,100,100,100,100,100,100,100,100,100,...,100,-7541.264300,4.864921e+06,2,1,106,2,2,23,1371713733
1,100,100,100,100,100,100,100,100,100,100,...,100,-7536.621200,4.864934e+06,2,1,106,2,2,23,1371713691
2,100,100,100,100,100,100,100,-97,100,100,...,100,-7519.152400,4.864950e+06,2,1,103,2,2,23,1371714095
3,100,100,100,100,100,100,100,100,100,100,...,100,-7524.570400,4.864934e+06,2,1,102,2,2,23,1371713807
4,100,100,100,100,100,100,100,100,100,100,...,100,-7632.143600,4.864982e+06,0,0,122,2,11,13,1369909710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1106,100,100,100,100,100,100,100,100,100,100,...,100,-7317.344231,4.864796e+06,3,2,0,0,0,13,1381156711
1107,100,100,100,100,100,100,100,100,100,100,...,100,-7313.731120,4.864792e+06,3,2,0,0,0,13,1381156730
1108,100,100,100,100,100,100,100,100,100,100,...,100,-7637.535798,4.864903e+06,0,0,0,0,0,13,1381247781
1109,100,100,100,100,100,100,100,100,100,100,...,100,-7636.654005,4.864905e+06,0,0,0,0,0,13,1381247807


In [ ]:
df.isnull().sum().any()

False

In [ ]:
wap_cols = [col for col in df.columns if 'WAP' in col]

In [ ]:
# # df[wap_cols] = df[wap_cols].replace(100, -1000)

# df_train[wap_cols] = df_train[wap_cols].replace(100, -1000)
# df_val[wap_cols] = df_val[wap_cols].replace(100, -1000)

In [ ]:
df_train['CLASS'] = df_train['BUILDINGID'].astype(str) + df_train['FLOOR'].astype(str)
df_val['CLASS'] = df_val['BUILDINGID'].astype(str) + df_val['FLOOR'].astype(str)

In [ ]:
# Encode labels
enc = LabelEncoder()
y_train = enc.fit_transform(df_train[['CLASS']].values.reshape(-1, 1)).reshape(-1,)
y_test = enc.transform(df_val[['CLASS']].values.reshape(-1, 1)).reshape(-1,)

# enc = LabelEncoder()
# y_train = enc.fit_transform(df_train[['location']].values.reshape(-1, 1)).reshape(-1,)
# y_test = enc.transform(df_val[['location']].values.reshape(-1, 1)).reshape(-1,)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


In [ ]:
# Select features
df_x_train = df_train.iloc[:, :520]
df_x_val = df_val.iloc[:, :520]

# # Select features
# df_x_train = df_train.iloc[:, :188]
# df_x_val = df_val.iloc[:, :188]

In [ ]:
# # Normalizing the training and validation sets separately
# norm_scl = StandardScaler()
# norm_scl.fit(df_x_train)

# X_train = norm_scl.transform(df_x_train)
# X_test = norm_scl.transform(df_x_val)

X_train = df_x_train
X_test = df_x_val

In [ ]:
knn = KNeighborsClassifier()

    # Train the model on the training set
knn.fit(X_train, y_train)

    # Predict on the test set
y_pred = knn.predict(X_test)

acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')

print("Accuracy score: {:.4f}".format(acc))
print("Precision score: {:.4f}".format(prec))
print("Recall score: {:.4f}".format(recall))

Accuracy score: 0.7291
Precision score: 0.7891
Recall score: 0.7291


In [ ]:
support_vector_classifier = SVC()

    # Train the model on the training set
support_vector_classifier.fit(X_train, y_train)

    # Predict on the test set
y_pred = support_vector_classifier.predict(X_test)

acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')

print("Accuracy score: {:.4f}".format(acc))
print("Precision score: {:.4f}".format(prec))
print("Recall score: {:.4f}".format(recall))

Accuracy score: 0.8083
Precision score: 0.8317
Recall score: 0.8083


In [ ]:
support_vector_classifier = RandomForestClassifier()

    # Train the model on the training set
support_vector_classifier.fit(X_train, y_train)

    # Predict on the test set
y_pred = support_vector_classifier.predict(X_test)

acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')

print("Accuracy score: {:.4f}".format(acc))
print("Precision score: {:.4f}".format(prec))
print("Recall score: {:.4f}".format(recall))

Accuracy score: 0.8695
Precision score: 0.8967
Recall score: 0.8695


In [ ]:
from sklearn.model_selection import GridSearchCV, train_test_split

# Define the parameter grid
param_grid = {'n_neighbors': range(1, 31)}

# Initialize the KNN classifier
knn = RandomForestClassifier()

# Sample sizes
sample_sizes = [0.2, 0.4, 0.6, 0.8, 1.0]

# Iterate over different sample sizes
for sample_size in sample_sizes:
    print(f"\nSample size: {sample_size}")

    # Sample the data
    if sample_size < 1.0:
        # Sample the data
        X_sampled, _, y_sampled, _ = train_test_split(X_train, y_train, train_size=sample_size, stratify=y_train, random_state=42)
    else:
        # Use the entire dataset
        X_sampled, y_sampled = X_train, y_train

    # Initialize GridSearchCV

    # Fit the grid search to the sampled data
    knn.fit(X_sampled, y_sampled)

    # Get the best estimator

    y_pred = knn.predict(X_test)

    # Evaluate the model
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')

    print("Accuracy score: {:.4f}".format(acc))
    print("Precision score: {:.4f}".format(prec))
    print("Recall score: {:.4f}".format(recall))


Sample size: 0.2
Accuracy score: 0.8533
Precision score: 0.8823
Recall score: 0.8533

Sample size: 0.4
Accuracy score: 0.8605
Precision score: 0.8867
Recall score: 0.8605

Sample size: 0.6
Accuracy score: 0.8740
Precision score: 0.8998
Recall score: 0.8740

Sample size: 0.8
Accuracy score: 0.8749
Precision score: 0.9025
Recall score: 0.8749

Sample size: 1.0
Accuracy score: 0.8722
Precision score: 0.8976
Recall score: 0.8722


In [ ]:
from sklearn.model_selection import GridSearchCV, train_test_split

# Define the parameter grid
param_grid = {'n_neighbors': range(1, 31)}

# Initialize the KNN classifier
knn = KNeighborsClassifier()

# Sample sizes
sample_sizes = [0.2, 0.4, 0.6, 0.8, 1.0]

# Iterate over different sample sizes
for sample_size in sample_sizes:
    print(f"\nSample size: {sample_size}")

    # Sample the data
    if sample_size < 1.0:
        # Sample the data
        X_sampled, _, y_sampled, _ = train_test_split(X_train, y_train, train_size=sample_size, stratify=y_train, random_state=42)
    else:
        # Use the entire dataset
        X_sampled, y_sampled = X_train, y_train

    # Initialize GridSearchCV
    grid_search = GridSearchCV(estimator=knn, param_grid=param_grid, cv=5, scoring='accuracy')

    # Fit the grid search to the sampled data
    grid_search.fit(X_sampled, y_sampled)

    # Get the best estimator
    best_knn = grid_search.best_estimator_

    # Predict on the test set using the best estimator
    y_pred = best_knn.predict(X_test)

    # Evaluate the model
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')

    print("Best number of neighbors: {}".format(grid_search.best_params_['n_neighbors']))
    print("Accuracy score: {:.4f}".format(acc))
    print("Precision score: {:.4f}".format(prec))
    print("Recall score: {:.4f}".format(recall))


Sample size: 0.2
Best number of neighbors: 1
Accuracy score: 0.7435
Precision score: 0.7990
Recall score: 0.7435

Sample size: 0.4
Best number of neighbors: 1
Accuracy score: 0.7552
Precision score: 0.8051
Recall score: 0.7552

Sample size: 0.6
Best number of neighbors: 1
Accuracy score: 0.7579
Precision score: 0.8044
Recall score: 0.7579

Sample size: 0.8
Best number of neighbors: 1
Accuracy score: 0.7642
Precision score: 0.8044
Recall score: 0.7642

Sample size: 1.0
Best number of neighbors: 1
Accuracy score: 0.7696
Precision score: 0.8055
Recall score: 0.7696


In [ ]:
# Define the parameter grid for C values
param_grid = {'C': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5]}


# Initialize the Support Vector Classifier
svc = SVC()

# Sample sizes
sample_sizes = [0.2, 0.4, 0.6, 0.8, 1.0]

# Iterate over different sample sizes
for sample_size in sample_sizes:
    print(f"\nSample size: {sample_size}")

    # Sample the data
    if sample_size < 1.0:
        # Sample the data
        X_sampled, _, y_sampled, _ = train_test_split(X_train, y_train, train_size=sample_size, stratify=y_train, random_state=42)
    else:
        # Use the entire dataset
        X_sampled, y_sampled = X_train, y_train

    # Initialize GridSearchCV
    grid_search = GridSearchCV(estimator=svc, param_grid=param_grid, cv=5, scoring='accuracy')

    # Fit the grid search to the sampled data
    grid_search.fit(X_sampled, y_sampled)

    # Get the best estimator
    best_svc = grid_search.best_estimator_

    # Predict on the test set using the best estimator
    y_pred = best_svc.predict(X_test)

    # Evaluate the model
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')

    print("Best number of neighbors: {}".format(grid_search.best_params_['C']))
    print("Accuracy score: {:.4f}".format(acc))
    print("Precision score: {:.4f}".format(prec))
    print("Recall score: {:.4f}".format(recall))


Sample size: 0.2
Best number of neighbors: 1.5
Accuracy score: 0.8416
Precision score: 0.8663
Recall score: 0.8416

Sample size: 0.4
Best number of neighbors: 1.3
Accuracy score: 0.8425
Precision score: 0.8666
Recall score: 0.8425

Sample size: 0.6
Best number of neighbors: 1.5
Accuracy score: 0.8398
Precision score: 0.8629
Recall score: 0.8398

Sample size: 0.8
Best number of neighbors: 1.4
Accuracy score: 0.8461
Precision score: 0.8646
Recall score: 0.8461

Sample size: 1.0
Best number of neighbors: 1.4
Accuracy score: 0.8452
Precision score: 0.8625
Recall score: 0.8452


In [ ]:
# Define the parameter grid for MLP
param_grid = {
    'learning_rate_init': [1e-3, 1e-4],
    'batch_size': [16, 32, 64, 128],
    'alpha': [1e-3, 1e-4],  # 'alpha' is the parameter for weight decay in MLPClassifier
}

# Initialize the MLP Classifier
mlp = MLPClassifier(max_iter=1000)

# Sample sizes
sample_sizes = [0.2, 0.4, 0.6, 0.8, 1.0]

# Iterate over different sample sizes
for sample_size in sample_sizes:
    print(f"\nSample size: {sample_size}")

    # Sample the data
    if sample_size < 1.0:
        # Sample the data
        X_sampled, _, y_sampled, _ = train_test_split(X_train, y_train, train_size=sample_size, stratify=y_train, random_state=42)
    else:
        # Use the entire dataset
        X_sampled, y_sampled = X_train, y_train

    # Initialize GridSearchCV
    grid_search = GridSearchCV(estimator=mlp, param_grid=param_grid, cv=5, scoring='accuracy')

    # Fit the grid search to the sampled data
    grid_search.fit(X_sampled, y_sampled)

    # Get the best estimator
    best_mlp = grid_search.best_estimator_

    # Predict on the test set using the best estimator
    y_pred = best_mlp.predict(X_test)

    # Evaluate the model
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')


    print("Best parameters: {}".format(grid_search.best_params_))
    print("Accuracy score: {:.4f}".format(acc))
    print("Precision score: {:.4f}".format(prec))
    print("Recall score: {:.4f}".format(recall))



Sample size: 0.2
Best parameters: {'alpha': 0.0001, 'batch_size': 32, 'learning_rate_init': 0.0001}
Accuracy score: 0.7408
Precision score: 0.7703
Recall score: 0.7408

Sample size: 0.4
Best parameters: {'alpha': 0.0001, 'batch_size': 16, 'learning_rate_init': 0.0001}
Accuracy score: 0.7390
Precision score: 0.8029
Recall score: 0.7390

Sample size: 0.6
Best parameters: {'alpha': 0.0001, 'batch_size': 32, 'learning_rate_init': 0.0001}
Accuracy score: 0.7813
Precision score: 0.8180
Recall score: 0.7813

Sample size: 0.8
Best parameters: {'alpha': 0.001, 'batch_size': 16, 'learning_rate_init': 0.0001}
Accuracy score: 0.8191
Precision score: 0.8356
Recall score: 0.8191

Sample size: 1.0


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:693: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


In [ ]:

# Combine normalized training and test sets for tensor conversion
df_x = np.concatenate((X_train, X_test), axis=0)
y = np.concatenate((y_train, y_test), axis=0)

In [ ]:

# Convert to tensors
x = torch.tensor(df_x, dtype=torch.float).float()
y = torch.tensor(y, dtype=torch.float).long().reshape(-1)

# Create the Data object
data = Data(x=x, y=y)
data = data.to(device)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
# Define masks
train_mask = torch.BoolTensor([True] * train_numb + [False] * test_numb).view(-1)
test_mask = torch.BoolTensor([False] * train_numb + [True] * test_numb).view(-1)
data.train_mask = torch.tensor(train_mask, dtype=torch.bool)
data.test_mask = torch.tensor(test_mask, dtype=torch.bool)

<ipython-input-46-adfa66ee5378>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data.train_mask = torch.tensor(train_mask, dtype=torch.bool)
<ipython-input-46-adfa66ee5378>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data.test_mask = torch.tensor(test_mask, dtype=torch.bool)


In [ ]:
# data.train_mask = torch.tensor(train_mask, dtype=torch.bool)
# data.test_mask = torch.tensor(test_mask, dtype=torch.bool)

In [ ]:
 # number of nodes
print("Number of nodes: ", data.num_nodes)

Number of nodes:  10469


In [ ]:
 # sample nodes from the graph
print("Shape of sample nodes: ", data.x.shape)

Shape of sample nodes:  torch.Size([10469, 188])


In [ ]:
# check training nodes
print("# of nodes to train on: ", data.train_mask.sum().item())

# check test nodes
print("# of nodes to test on: ", data.test_mask.sum().item())

# of nodes to train on:  8375
# of nodes to test on:  2094


In [ ]:
print("X shape: ", data.x.shape)
# print("Edge shape: ", data.edge_index.shape)
print("Y shape: ", data.y.shape)

X shape:  torch.Size([10469, 188])
Y shape:  torch.Size([10469])


## DYNAMICEDGECONV


In [ ]:
class EdgeConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super().__init__(aggr='add')
        # self.mlp = Seq(Linear(2 * in_channels, out_channels), BatchNorm1d(out_channels), ReLU(), Linear(out_channels, out_channels), BatchNorm1d(out_channels), ReLU())
        self.mlp = Seq(Linear(2 * in_channels, out_channels), ReLU(), BatchNorm1d(out_channels), Linear(out_channels, out_channels), ReLU(), BatchNorm1d(out_channels), Linear(out_channels, out_channels))

    def forward(self, x, edge_index):
        return self.propagate(edge_index, x=x)

    def message(self, x_i, x_j):
        tmp = torch.cat([x_i, x_j - x_i], dim=-1)  # tmp has shape [E, 2 * in_channels]
        return self.mlp(tmp)

In [ ]:
class DynamicEdgeConv(EdgeConv):
    def __init__(self, in_channels, out_channels, k=5):
        super().__init__(in_channels, out_channels)
        self.k = k

    def forward(self, x, batch=None):
        edge_index = knn_graph(x, self.k, batch, loop=False, flow=self.flow)
        return super().forward(x, edge_index)

In [ ]:
# class DEC(torch.nn.Module):
#     def __init__(self, k1, k2):
#         super().__init__()
#         self.conv1 = DynamicEdgeConv(data.num_features, 256, k=k1)
#         self.conv2 = DynamicEdgeConv(256, 50, k=k2)
#         self.fc1 = nn.Linear(50, 13)
#         self.dropout = nn.Dropout(p=0.5)

#     def forward(self, data):
#         x = data.x
#         x = self.conv1(x)
#         x = F.relu(x)
#         x = self.dropout(x)
#         x = self.conv2(x)
#         x = F.relu(x)
#         x = self.dropout(x)
#         x = self.fc1(x)

#         return x

class DEC(torch.nn.Module):
    def __init__(self, k1, k2):
        super().__init__()
        self.conv1 = DynamicEdgeConv(data.num_features, 256, k=k1)
        self.conv2 = DynamicEdgeConv(256, 64, k=k2)
        self.fc1 = nn.Linear(64, 32)
        self.fc2 = nn.Linear(32, 16)
        self.dropout = nn.Dropout(p=0.5)
        self.batch_norm1 = BatchNorm1d(256)
        self.batch_norm2 = BatchNorm1d(64)
        self.batch_norm_fc = BatchNorm1d(32)

    def forward(self, data):
        x = data.x
        x = self.conv1(x)
        x = self.batch_norm1(x)
        x = F.leaky_relu(x)
        x = self.dropout(x)

        x = self.conv2(x)
        x = self.batch_norm2(x)
        x = F.leaky_relu(x)
        x = self.dropout(x)

        x = self.fc1(x)
        x = self.batch_norm_fc(x)
        x = F.leaky_relu(x)
        x = self.dropout(x)

        x = self.fc2(x)
        return x

In [ ]:
def compute_accuracy(pred_y, y):
    return (pred_y == y).sum().item()

In [ ]:
def train_fun(k1, k2, lr, device):
    model = DEC(k1, k2).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=5e-4)
    loss_function = nn.CrossEntropyLoss()

    losses = []
    accuracies = []
    test_accuracies = []

    # epoch = 0
    for epoch in range(500):
        model.train()
        optimizer.zero_grad()
        out = model(data)

        loss = loss_function(out[data.train_mask], data.y[data.train_mask])
        # correct = compute_accuracy(out.argmax(dim=1)[data.train_mask], data.y[data.train_mask])

        correct = int(out.argmax(dim=1)[data.train_mask].eq(data.y[data.train_mask]).sum().item())
        acc = correct / int(data.train_mask.sum())
        losses.append(loss.item())
        accuracies.append(acc)

        loss.backward()
        optimizer.step()

        # epoch+=1

        if (epoch) % 1 == 0:
            model.eval()
            with torch.no_grad():
                _, pred = model(data).max(dim=1)
                # correct_test = compute_accuracy(pred[data.test_mask], data.y[data.test_mask])
                correct_test = int(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
                acc_test = correct_test / int(data.test_mask.sum())
                test_accuracies.append(acc_test)
                print(f'Epoch: {epoch}, \tTraining Accuracy: {acc}, \tTest Accuracy: {acc_test}')

    res = f"Best Training Accuracy: {max(accuracies)}, Best Test Accuracy: {max(test_accuracies)}"
    print(res)
    plt.plot(accuracies, label='Train Accuracy')
    plt.plot(test_accuracies, label='Test Accuracy')
    plt.title("Training and Test Accuracy over Epochs")
    plt.legend()
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    # plt.savefig(f'./gravnet_training_results.png')
    plt.show()


In [ ]:
for k1 in [30, 20, 15, 10, 5, 2]:
    for k2 in [15, 10, 6, 3, 1]:
        if k1 > k2:
            print("Doing for: ", k1, k2)
            train_fun(k1, k2, 0.009, device)

Doing for:  30 15


KeyboardInterrupt: 

In [ ]:
from torch_geometric.nn import GravNetConv

In [ ]:
# class GravNetConv(MessagePassing):
#     def __init__(self, in_channels, out_channels, space_dimensions, propagate_dimensions, k=5):
#         super().__init__(aggr='add')  # 'add' aggregation.
#         self.space_dimensions = space_dimensions
#         self.propagate_dimensions = propagate_dimensions
#         self.k = k
#         # Layers for transforming the feature space into a low-dimensional space where nearest neighbors are calculated.
#         self.linear_space = Linear(in_channels, space_dimensions) #lin_s
#         # Layers for transforming the input features for message passing.
#         self.linear_features = Linear(in_channels, propagate_dimensions) #lin_h
#         # MLP for updating the features after message passing.
#         self.mlp = Seq(
#             Linear(2 * propagate_dimensions, out_channels),
#             ReLU(),
#             Linear(out_channels, out_channels)
#         )
#         # Batch normalization layers.
#         self.bn_space = BatchNorm1d(space_dimensions)
#         self.bn_features = BatchNorm1d(propagate_dimensions)

#     def forward(self, x, batch=None):
#         # Transform to space in which we compute nearest neighbors.
#         space = self.bn_space(self.linear_space(x)) #s_l

#         # Compute the graph dynamically based on nearest neighbors in the "space".
#         edge_index = knn_graph(space, self.k, batch, loop=False, flow=self.flow)

#         # Transform features for message passing.
#         x_features = self.bn_features(self.linear_features(x)) #h_l

#         # Propagate the messages using the dynamically computed graph.
#         return self.propagate(edge_index=edge_index, x=x_features, space=space)

#     def message(self, x_i, x_j, space_i, space_j):
#         # Compute distance in the space dimension for weighting.
#         dist = (space_i - space_j).pow(2).sum(dim=-1).sqrt()
#         # weight = softmax(dist, index=self.edge_index[0])

#         # Concatenate the features of node i and weighted features from node j.
#         # tmp = torch.cat([x_i, weight * (x_j - x_i)], dim=1)
#         tmp = torch.cat([x_i, x_j - x_i], dim=1)  # Direct concatenation without using weights


#         return self.mlp(tmp)

In [ ]:
class GravNetConv(MessagePassing):
    def __init__(self, in_channels, out_channels, space_dimensions, propagate_dimensions, k=5):
        super().__init__(aggr='mean')  # 'add' aggregation.
        self.space_dimensions = space_dimensions
        self.propagate_dimensions = propagate_dimensions
        self.k = k
        # Layers for transforming the feature space into a low-dimensional space where nearest neighbors are calculated.
        self.linear_space = Linear(in_channels, space_dimensions) #lin_s
        # Layers for transforming the input features for message passing.
        self.linear_features = Linear(in_channels, propagate_dimensions) #lin_h
        # MLP for updating the features after message passing.
        self.lin_out1 = Linear(in_channels, out_channels, bias=False)
        self.lin_out2 = Linear(propagate_dimensions, out_channels)
        # # Batch normalization layers.
        # self.bn_space = BatchNorm1d(space_dimensions)
        # self.bn_features = BatchNorm1d(propagate_dimensions)

    def forward(self, x, batch=None):
        # Transform to space in which we compute nearest neighbors.

        x = (x,x)

        space_l = self.linear_space(x[0]) #s_l
        space_r = self.linear_space(x[1])

        # Compute the graph dynamically based on nearest neighbors in the "space".
        edge_index = knn(space_l, space_r, self.k, batch).flip([0])

        edge_weight = (space_l[edge_index[0]] - space_r[edge_index[1]]).pow(2).sum(dim=-1)
        edge_weight = torch.exp(-10 * edge_weight)

        # Transform features for message passing.
        x_features = self.linear_features(x[0]) #h_l

        # Propagate the messages using the dynamically computed graph.
        out = self.propagate(edge_index=edge_index, x=(x_features,None), edge_weight=edge_weight, size=(space_l.size(0), space_r.size(0)))

        return self.lin_out1(x[1]) + self.lin_out2(out)

    def message(self, x_j, edge_weight):

        return x_j * edge_weight.unsqueeze(1)

In [ ]:
# class CustomGravNet(torch.nn.Module):
#     def __init__(self, space_dimensions, propagate_dimensions, k1, k2, k3, k4):
#         super().__init__()
#         self.conv1 = GravNetConv(data.num_features, 512, space_dimensions, propagate_dimensions, k=k1)
#         self.conv2 = GravNetConv(512, 256, space_dimensions, propagate_dimensions, k=k2)
#         self.conv3 = GravNetConv(256, 128, space_dimensions, propagate_dimensions, k=k3)
#         self.conv4 = GravNetConv(128, 64, space_dimensions, propagate_dimensions, k=k4)

#         self.fc1 = Linear(64, 32)
#         self.fc2 = Linear(32, 13)

#         self.dropout = nn.Dropout(p=0.5)

#         self.batch_norm1 = BatchNorm1d(512)
#         self.batch_norm2 = BatchNorm1d(256)
#         self.batch_norm3 = BatchNorm1d(128)
#         self.batch_norm4 = BatchNorm1d(64)

#         self.batch_norm_fc = BatchNorm1d(32)

#     def forward(self, data):
#         x = data.x
#         x = self.conv1(x)
#         x = self.batch_norm1(x)
#         x = F.leaky_relu(x)
#         x = self.dropout(x)

#         x = self.conv2(x)
#         x = self.batch_norm2(x)
#         x = F.leaky_relu(x)
#         x = self.dropout(x)

#         x = self.conv3(x)
#         x = self.batch_norm3(x)
#         x = F.leaky_relu(x)
#         x = self.dropout(x)

#         x = self.conv4(x)
#         x = self.batch_norm4(x)
#         x = F.leaky_relu(x)
#         x = self.dropout(x)

#         x = self.fc1(x)
#         x = self.batch_norm_fc(x)
#         x = F.leaky_relu(x)
#         x = self.dropout(x)

#         x = self.fc2(x)
#         return x

In [ ]:
class CustomGravNet(torch.nn.Module):
    def __init__(self, space_dimensions, propagate_dimensions, k1, k2):
        super().__init__()
        self.conv1 = GravNetConv(data.num_features, 256, space_dimensions, propagate_dimensions, k=k1)
        self.conv2 = GravNetConv(256, 64, space_dimensions, propagate_dimensions, k=k2)
        # self.conv3 = GravNetConv(256, 128, space_dimensions, propagate_dimensions, k=k3)
        # self.conv4 = GravNetConv(128, 64, space_dimensions, propagate_dimensions, k=k4)

        self.fc1 = Linear(64, 32)
        self.fc2 = Linear(32, 16)

        self.dropout = nn.Dropout(p=0.5)

        self.batch_norm1 = BatchNorm1d(256)
        self.batch_norm2 = BatchNorm1d(64)
        # self.batch_norm3 = BatchNorm1d(128)
        # self.batch_norm4 = BatchNorm1d(64)

        self.batch_norm_fc = BatchNorm1d(32)

    def forward(self, data):
        x = data.x
        x = self.conv1(x)
        x = self.batch_norm1(x)
        x = F.leaky_relu(x)
        x = self.dropout(x)

        x = self.conv2(x)
        x = self.batch_norm2(x)
        x = F.leaky_relu(x)
        x = self.dropout(x)

        # x = self.conv3(x)
        # x = self.batch_norm3(x)
        # x = F.leaky_relu(x)
        # x = self.dropout(x)

        # x = self.conv4(x)
        # x = self.batch_norm4(x)
        # x = F.leaky_relu(x)
        # x = self.dropout(x)

        x = self.fc1(x)
        x = self.batch_norm_fc(x)
        x = F.leaky_relu(x)
        x = self.dropout(x)

        x = self.fc2(x)
        return x

In [ ]:
data.x.shape

torch.Size([10469, 188])

In [ ]:
def compute_accuracy(pred_y, y):
    return (pred_y == y).sum().item()

In [ ]:


def train_gravnet(k1, k2, lr, data, device):
    model = CustomGravNet(space_dimensions=4, propagate_dimensions=22, k1=k1, k2=k2).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=5e-4)
    loss_function = torch.nn.CrossEntropyLoss()

    losses = []
    accuracies = []
    test_accuracies = []

    # epoch = 0
    for epoch in range(500):
        model.train()
        optimizer.zero_grad()
        out = model(data)

        loss = loss_function(out[data.train_mask], data.y[data.train_mask])
        # correct = compute_accuracy(out.argmax(dim=1)[data.train_mask], data.y[data.train_mask])

        correct = int(out.argmax(dim=1)[data.train_mask].eq(data.y[data.train_mask]).sum().item())
        acc = correct / int(data.train_mask.sum())
        losses.append(loss.item())
        accuracies.append(acc)

        loss.backward()
        optimizer.step()

        # epoch+=1

        if (epoch) % 1 == 0:
            model.eval()
            with torch.no_grad():
                _, pred = model(data).max(dim=1)
                # correct_test = compute_accuracy(pred[data.test_mask], data.y[data.test_mask])
                correct_test = int(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
                acc_test = correct_test / int(data.test_mask.sum())
                test_accuracies.append(acc_test)
                print(f'Epoch: {epoch}, \tTraining Accuracy: {acc}, \tTest Accuracy: {acc_test}')

    res = f"Best Training Accuracy: {max(accuracies)}, Best Test Accuracy: {max(test_accuracies)}"
    print(res)
    plt.plot(accuracies, label='Train Accuracy')
    plt.plot(test_accuracies, label='Test Accuracy')
    plt.title("Training and Test Accuracy over Epochs")
    plt.legend()
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    # plt.savefig(f'./gravnet_training_results.png')
    plt.show()


In [ ]:
for k1 in [10, 9, 8, 7, 6, 5, 4, 3]:
    for k2 in [8,7,6,5,4,3,2,1]:
      if k1 > k2:
          print("Doing for: ", k1, k2)
          train_gravnet(k1, k2, lr=0.009, data=data, device=device)


Doing for:  10 8


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
